In [2]:
import cv2
import numpy as np
import pandas as pd
import joblib
import mediapipe as mp

###### Chargement de modèle entrainé avec joblib

In [3]:
svm_model_loaded = joblib.load('asl_svm_model.pkl')

###### Extraction et représentation des landmarks de la main sur l'image.

In [4]:
# extracting the land marks of the hand
def extract_landmarks(image, results, mpDraw, mpHands):
    # Extraction des dimensions de l'image
    h, w, c = image.shape
    # Initialisation du tableau pour stocker les coordonnées des landmarks
    landmarks_array = []
    
    # Vérification de la présence de landmarks pour plusieurs mains dans les résultats
    if results.multi_hand_landmarks:
        # Boucle à travers les landmarks de chaque main détectée
        for hand_landmarks in results.multi_hand_landmarks:
            
            # Extraction des coordonnées des landmarks
            landmarks = hand_landmarks.landmark
            landmarks_row = []
            
            # Boucle à travers chaque landmark
            for landmark in landmarks:
                # Stockage des coordonnées des landmarks dans le tableau
                landmarks_row.extend([landmark.x*w, landmark.y*h, landmark.z*c])
                
                # Dessin d'un cercle pour représenter chaque landmark sur l'image
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(image, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                
            # Stockage des coordonnées des landmarks de la main dans le tableau principal
            landmarks_array.append(landmarks_row)
        
            # Dessin des connexions entre les landmarks de la main sur l'image
            mpDraw.draw_landmarks(image, hand_landmarks, mpHands.HAND_CONNECTIONS)
        
        # Traitement du nombre de mains détectées
        vector_landmarks = []
        nom_hands = len(results.multi_hand_landmarks)
        
        # Ajout d'informations supplémentaires au tableau des landmarks en fonction du nombre de mains détectées
        if nom_hands == 1:
            landmarks_array[0].append(0)
            landmarks_array.append([0] * 63)
        elif nom_hands == 2:
            landmarks_array[0].append(1)
            
        # Retour de l'image avec les landmarks dessinés et des coordonnées des landmarks
        return image,np.array([ landmarks_array[0]+landmarks_array[1] ]).flatten()
    else:
        # Si aucun landmark n'est détecté, retourne None
        return None

###### Configuration de la capture vidéo de la webcam

In [5]:
# Capture vidéo en direct à partir de la webcam (index 0)
cap = cv2.VideoCapture(0)

# Taille désirée pour la webcam
webcam_width, webcam_height = 400, 400

# # Ajustement de la taille de la webcam
cap.set(cv2.CAP_PROP_FRAME_WIDTH, webcam_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, webcam_height)

True

###### Classification en temps réel des gestes de la main

In [6]:
mpHands = mp.solutions.hands  # Importation du modèle de détection des mains
hands = mpHands.Hands()  # Initialisation du détecteur de mains
mpDraw = mp.solutions.drawing_utils  # Utilitaire pour dessiner les landmarks des mains

while True:
    ret, frame = cap.read()  # Lecture d'une image de la webcam
    
    if not ret:
        break  # Sortir de la boucle si la lecture de la webcam échoue
        
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8))  # Traitement de l'image pour détecter les mains

    if results.multi_hand_landmarks:  # Si des landmarks de main sont détectés
        frame_marked, landmarks = extract_landmarks(frame, results, mpDraw, mpHands)  # Extraction et dessin des landmarks de la main

        landmarks_dataframe = pd.DataFrame([landmarks])  # Conversion des landmarks en DataFrame
        class_ = svm_model_loaded.predict(landmarks_dataframe)  # Prédiction de la classe du geste de la main
        
        # Affichage de la classe prédite sur l'image
        cv2.putText(frame_marked, class_[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        cv2.imshow('Classification en temps réel', frame_marked)  # Affichage de l'image avec les landmarks de la main et la classe prédite
    
    else:
        cv2.imshow('Classification en temps réel', frame)  # Affichage de l'image de la webcam si aucun landmark de main n'est détecté
    
    # Quitter la boucle si la touche 'q' est enfoncée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libération des ressources
cap.release()
cv2.destroyAllWindows()

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

...

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\mohammed\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with

In [7]:
# Libérer la capture vidéo et fermer les fenêtres OpenCV
cap.release()
cv2.destroyAllWindows()